In [268]:
%run func_def.ipynb

**imports loaded.

> File management func:
    get_content(directory)

> Pre-Processing func:
    get_striking_efficiency(name, date)
    get_height(item)
    get_weight(item)
    get_reach(item)
    get_std_reach(item, height, reach_list, height_list)
    get_age(item)
    get_std_age(item, age_list)
    get_win_prob(name, date, df)
    get_win_streak(name, date, df) #return count of recent wins
    get_lose_streak(name, date, df)
    get_fighter_stats(name, data )

> Feature Engineering func:
    get_stat(index, data, stat='height', fo='')
    get_swaped_row(index, data) # return fighter & opponent statistics swapped. 
    get_wins(combo, data)
    get_wins_ratio(combo, data)

> outlier_cleaning func:
    set_outlier_detection_model(X_columns, data)

> model_stats_pred_training func:
    set_lower_cases(string)
    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)
    set_xgboost_reg(X_columns, y_columns, data, n=200, learning_rate=0.2, subsample=0.4, max_dept

### <span style='color:green'> **[done]** </span>Loading Feature's-ready data

In [269]:
loc = '../data/'
file = 'data_features_outliers_detected.csv'
df = pd.read_csv(loc+file)
df.tail(3)

,result,fighter,opponent,date,weight_class,method,type,kd_dif,str_dif,td_dif,sub_dif,td_avg_dif,td_def_dif,sub_avg_dif,slpm_dif,str_acc_dif,sapm_dif,str_def_dif,str_efc_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,outlier,lof_outlier
6391,lose,Jamie Mullarkey,Muhammad Naimov,2023-06-03,Lightweight,KO/TKO,Punch,-1.341659,0.384238,1.182042,-0.008752,1.633408,0.864923,0.2,0.437681,0.473642,-0.833962,0.849897,0.00651,0.232252,1.412053,1.122819,-0.018188,False,False
6392,win,Elise Reed,Jinh Yu Frey,2023-06-03,Women's Strawweight,U-DEC,Unanimous,0.004210,0.899464,-1.175651,-0.008752,-0.199954,-1.345974,-0.3,-0.013941,0.638938,-0.606841,0.066464,0.00758,-1.880897,0.248877,0.009144,-0.018188,False,False
6393,lose,Luan Lacerda,Da'Mon Blackshear,2023-06-03,Bantamweight,KO/TKO,Punches,0.004210,-1.419052,0.003196,0.852188,0.057950,0.726742,-0.2,0.581949,0.060402,2.049208,-0.368777,0.00000,-1.259599,0.001393,0.009144,-0.018188,False,False


In [270]:
X = ['td_avg_dif','td_def_dif','sub_avg_dif',
     'slpm_dif','str_acc_dif','sapm_dif','str_def_dif','str_efc_dif',
     'stance_win_ratio','win_perc_dif','win_strk_dif','lose_strk_dif']
y = 'result'

DIM = len(X)
NEURON_DENSE = 128

print(f'X = {X}')
print(f'Y = {y}')

X = ['td_avg_dif', 'td_def_dif', 'sub_avg_dif', 'slpm_dif', 'str_acc_dif', 'sapm_dif', 'str_def_dif', 'str_efc_dif', 'stance_win_ratio', 'win_perc_dif', 'win_strk_dif', 'lose_strk_dif']
Y = result


In [271]:
df = set_train_test(X_columns=X, y_columns=y, data=df, test_size=0.2, valid_size=0.2)
df.tail(3)

Data split: Data=(6394, 26), train_set=(4095, 12), validation_set=(971, 12), test_set=(1328, 12)


,set,result,fighter,opponent,date,weight_class,method,type,kd_dif,str_dif,td_dif,sub_dif,td_avg_dif,td_def_dif,sub_avg_dif,slpm_dif,str_acc_dif,sapm_dif,str_def_dif,str_efc_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,outlier,lof_outlier
6391,validation,lose,Jamie Mullarkey,Muhammad Naimov,2023-06-03,Lightweight,KO/TKO,Punch,-1.341659,0.384238,1.182042,-0.008752,1.633408,0.864923,0.2,0.437681,0.473642,-0.833962,0.849897,0.00651,0.232252,1.412053,1.122819,-0.018188,False,False
6392,train,win,Elise Reed,Jinh Yu Frey,2023-06-03,Women's Strawweight,U-DEC,Unanimous,0.004210,0.899464,-1.175651,-0.008752,-0.199954,-1.345974,-0.3,-0.013941,0.638938,-0.606841,0.066464,0.00758,-1.880897,0.248877,0.009144,-0.018188,False,False
6393,train,lose,Luan Lacerda,Da'Mon Blackshear,2023-06-03,Bantamweight,KO/TKO,Punches,0.004210,-1.419052,0.003196,0.852188,0.057950,0.726742,-0.2,0.581949,0.060402,2.049208,-0.368777,0.00000,-1.259599,0.001393,0.009144,-0.018188,False,False


In [272]:
df[y] = [1 if item == 'win' else 0 for item in df[y]] #replace win/lose with 1/0

X_train = df.loc[df.set=='train',X].values
X_test = df.loc[df.set=='test',X].values

y_train = df.loc[df.set=='train',y].values
y_test = df.loc[df.set=='test',y].values

In [273]:
model = keras.models.Sequential()

# Add layers to the model
model.add(keras.layers.Dense(NEURON_DENSE, activation='relu'))  # neurons in the hidden layer
model.add(keras.layers.Dense(NEURON_DENSE, activation='relu'))  # neurons in the hidden layer
model.add(keras.layers.Dense(1, activation='linear'))  # Output layer with 1 neuron f

In [274]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [275]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
128/128 [==============================] - 1s 1ms/step - loss: 0.8492 - accuracy: 0.5954
Epoch 2/5
128/128 [==============================] - 0s 1ms/step - loss: 0.7149 - accuracy: 0.6129
Epoch 3/5
128/128 [==============================] - 0s 1ms/step - loss: 0.6762 - accuracy: 0.6383
Epoch 4/5
128/128 [==============================] - 0s 1ms/step - loss: 0.7626 - accuracy: 0.6252
Epoch 5/5
128/128 [==============================] - 0s 947us/step - loss: 0.7168 - accuracy: 0.6037


In [276]:
X = df[X]
df['result_pred'] = model.predict(X)
df.tail(3)

200/200 [==============================] - 0s 595us/step


,set,result,fighter,opponent,date,weight_class,method,type,kd_dif,str_dif,td_dif,sub_dif,td_avg_dif,td_def_dif,sub_avg_dif,slpm_dif,str_acc_dif,sapm_dif,str_def_dif,str_efc_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,outlier,lof_outlier,result_pred
6391,validation,0,Jamie Mullarkey,Muhammad Naimov,2023-06-03,Lightweight,KO/TKO,Punch,-1.341659,0.384238,1.182042,-0.008752,1.633408,0.864923,0.2,0.437681,0.473642,-0.833962,0.849897,0.00651,0.232252,1.412053,1.122819,-0.018188,False,False,0.652664
6392,train,1,Elise Reed,Jinh Yu Frey,2023-06-03,Women's Strawweight,U-DEC,Unanimous,0.004210,0.899464,-1.175651,-0.008752,-0.199954,-1.345974,-0.3,-0.013941,0.638938,-0.606841,0.066464,0.00758,-1.880897,0.248877,0.009144,-0.018188,False,False,0.478126
6393,train,0,Luan Lacerda,Da'Mon Blackshear,2023-06-03,Bantamweight,KO/TKO,Punches,0.004210,-1.419052,0.003196,0.852188,0.057950,0.726742,-0.2,0.581949,0.060402,2.049208,-0.368777,0.00000,-1.259599,0.001393,0.009144,-0.018188,False,False,0.312012


In [277]:
df['prediction_dif'] = (df.result_pred - df.result).astype(int)
df.head()

,set,result,fighter,opponent,date,weight_class,method,type,kd_dif,str_dif,td_dif,sub_dif,td_avg_dif,td_def_dif,sub_avg_dif,slpm_dif,str_acc_dif,sapm_dif,str_def_dif,str_efc_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,outlier,lof_outlier,result_pred,prediction_dif
0,train,1,Max Holloway,Arnold Allen,2023-04-15,Featherweight,U-DEC,Unanimous,0.004210,2.592349,0.003196,-0.008752,-0.469071,0.277653,0.0,2.382163,0.556290,1.222742,-0.194681,-0.00454,-1.880897,-0.666814,-5.559232,1.001957,True,True,0.718244,0
1,train,1,Edson Barboza,Billy Quarantillo,2023-04-15,Featherweight,KO/TKO,Knee,1.350079,0.053021,0.003196,-0.008752,-0.631663,0.588561,-1.1,-2.435135,-1.179316,-1.048464,1.546283,0.00814,0.232252,-0.295587,-0.547693,2.022101,False,False,0.345140,0
2,test,1,Azamat Murzakanov,Dustin Jacoby,2023-04-15,Light Heavyweight,U-DEC,Unanimous,1.350079,0.089823,0.003196,-0.008752,0.198116,0.795832,0.0,-0.559651,0.804233,-0.568988,-0.107633,0.00399,1.159339,0.496362,0.565982,-1.038333,False,False,0.447678,0
3,train,0,Tanner Boser,Ion Cutelaba,2023-04-15,Light Heavyweight,KO/TKO,Punches,0.004210,-0.940628,-0.389753,-0.008752,-2.661255,-0.551433,0.0,-0.302477,0.638938,-0.512207,1.023994,0.00166,-1.880897,0.125135,0.009144,-2.058478,False,False,0.532514,0
4,test,1,Pedro Munhoz,Chris Gutierrez,2023-04-15,Bantamweight,U-DEC,Unanimous,1.350079,-0.646214,0.003196,-0.008752,0.074770,0.243108,0.5,0.331048,-1.261964,1.986119,-0.629922,-0.00292,0.232252,-1.161782,-3.331881,2.022101,True,False,0.426525,0


In [278]:
# model evaluation
train = df[df.set=='train']
test = df[df.set=='test']
train_corrects = len(train[train.prediction_dif==0])
train_false = len(train[train.prediction_dif!=0])
test_corrects = len(test[test.prediction_dif==0])
test_false = len(test[test.prediction_dif!=0])

print(f'Train Accuracy = {train_corrects/len(train)}')
print(f'Test Accuracy = {test_corrects/len(test)}')

Train Accuracy = 0.9968253968253968
Test Accuracy = 0.9969879518072289


In [ ]:
>>>>>>>>>>>

In [44]:
X_train = df[df['set'] == 'train'][X]
y_train = np.ravel(np.array(df[df['set'] == 'train'][y]).reshape(-1,1))
    
# Create a logistic regression model
model = LogisticRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Featherweight'